In [1]:
import pandas as pd
import re
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [2]:
def data(path):
    df = pd.read_csv(path, sep='>', lineterminator='>', 
                     header=None, squeeze=True).str.extract('\A(?P<accession>\S+)\s+(?P<sequence>[\S\n]+)\s*\Z', expand=True)
    df = df.applymap(lambda x: x.replace("\n", ""))
    df['len'] = df["sequence"].apply(len)
    return df.sort_values(by="len", ascending=False).reset_index(drop=True)
data("trim/Poil_contig.fa")

,accession,sequence,len
0,seq297_len158992_cov236,CGGCTTTAAAATCACGCGATGCATCAATAAACATCACAGAATCGTC...,158992
1,seq35_len105001_cov236,TTGTGGCTTTGCGTTCACCCTGCTATCTCCCGCCGCTCAACGCTAC...,105001
2,seq521_len100999_cov244,TTCAGTGACTGCGAAAGATGCAGCGGGTAATGAATCTCCAGCGACT...,100999
3,seq402_len91913_cov236,CAGAAGATAGCGATATCGGTATCGGTGACGTCAATCGCAAGCGTTG...,91913
4,seq548_len89248_cov219,AGCGAGCCGCCGCAAGAAGGTGATTGGTGTGGCGGAAGTCGAGCAG...,89248
...,...,...,...
616,seq2_len87_cov211,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,87
617,seq540_len87_cov463,CCTAAACAGGATATGCCTGAGGGAATCCCAATGCGTCTTACTGACT...,87
618,seq351_len87_cov463,CCCCATGCGCATCGGTCTTCTTCGCTTGAATTTCATCTGTATCAAA...,87
619,seq362_len87_cov295,GTGTGACATTGTAGGAGCGAAACTTTTTCGCGAATAGCGTCTCGGT...,87


In [3]:
cont = data("trim/Poil_contig.fa")
scaf = data("trim/Poil_scaffold.fa")
clos = data("trim/Poil_gapClosed.fa")

## Contigs

In [4]:
print("Всего контигов:", len(cont))

Всего контигов: 621


In [5]:
print("Размер самого длинного контига:", len(cont.iloc[0,1]))

Размер самого длинного контига: 158992


In [6]:
print("Общая длина:", cont['len'].sum())

Общая длина: 3926082


In [7]:
summ = 0
for elem in cont['len']:
    summ += elem
    if summ >= (cont['len'].sum() / 2):
        print("N50 =", elem)
        break

N50 = 50103


## Scaffolds

In [8]:
print("Всего скаффолдов:", len(scaf))

Всего скаффолдов: 99


In [9]:
print("Размер самого длинного скаффолда:", len(scaf.iloc[0,1]))

Размер самого длинного скаффолда: 383582


In [10]:
print("Общая длина:", scaf['len'].sum())

Общая длина: 3870080


In [11]:
summ = 0
for elem in scaf['len']:
    summ += elem
    if summ >= (scaf['len'].sum() / 2):
        print("N50 =", elem)
        break

N50 = 173397


In [12]:
print("Число гэпов:", len(re.findall("N+", scaf['sequence'][0])))

Число гэпов: 2


In [13]:
print("Суммарная длина гэпов:", scaf['sequence'][0].count("N"))

Суммарная длина гэпов: 27


## GapClosed

In [14]:
print("Всего скаффолдов:", len(clos))

Всего скаффолдов: 99


In [15]:
print("Размер самого длинного скаффолда:", len(clos.iloc[0,1]))

Размер самого длинного скаффолда: 383574


In [16]:
print("Общая длина:", clos['len'].sum())

Общая длина: 3870478


In [17]:
summ = 0
for elem in clos['len']:
    summ += elem
    if summ >= (clos['len'].sum() / 2):
        print("N50 =", elem)
        break

N50 = 173397


In [18]:
print("Число гэпов:", len(re.findall("N+", clos['sequence'][0])))

Число гэпов: 0


In [19]:
print("Суммарная длина гэпов:", clos['sequence'][0].count("N"))

Суммарная длина гэпов: 0


Сохраняем самый длинный скаффолд

In [20]:
seq = Seq(clos['sequence'][0])
seq_rec = SeqRecord(seq, id="The_longest_scaffold", description="")
SeqIO.write(seq_rec, "longest.fasta", "fasta")

1

Необязательная часть

Возьмём 2 млн paired-end ридов и 500 тыс. mate-pairs ридов

In [21]:
a_cont = data("add/trim/Poil_contig.fa")
a_scaf = data("add/trim/Poil_scaffold.fa")
a_clos = data("add/trim/Poil_gapClosed.fa")

In [22]:
# Контиги
print("Всего контигов:", len(a_cont))
print("Размер самого длинного контига:", len(a_cont.iloc[0,1]))
print("Общая длина:", a_cont['len'].sum())
summ = 0
for elem in a_cont['len']:
    summ += elem
    if summ >= (a_cont['len'].sum() / 2):
        print("N50 =", elem)
        break

Всего контигов: 725
Размер самого длинного контига: 235804
Общая длина: 3920791
N50 = 81379


In [23]:
# Скаффолды
print("Всего скаффолдов:", len(a_scaf))
print("Размер самого длинного скаффолда:", len(a_scaf.iloc[0,1]))
print("Общая длина:", a_scaf['len'].sum())
summ = 0
for elem in a_scaf['len']:
    summ += elem
    if summ >= (a_scaf['len'].sum() / 2):
        print("N50 =", elem)
        break
print("Число гэпов:", len(re.findall("N+", a_scaf['sequence'][0])))
print("Суммарная длина гэпов:", a_scaf['sequence'][0].count("N"))

Всего скаффолдов: 113
Размер самого длинного скаффолда: 383419
Общая длина: 3860910
N50 = 173286
Число гэпов: 6
Суммарная длина гэпов: 179


In [24]:
# Скаффолды после удаления гэпов
print("Всего скаффолдов:", len(a_clos))
print("Размер самого длинного скаффолда:", len(a_clos.iloc[0,1]))
print("Общая длина:", a_clos['len'].sum())
summ = 0
for elem in a_clos['len']:
    summ += elem
    if summ >= (a_clos['len'].sum() / 2):
        print("N50 =", elem)
        break
print("Число гэпов:", len(re.findall("N+", a_clos['sequence'][0])))
print("Суммарная длина гэпов:", a_clos['sequence'][0].count("N"))

Всего скаффолдов: 113
Размер самого длинного скаффолда: 383385
Общая длина: 3860685
N50 = 173286
Число гэпов: 2
Суммарная длина гэпов: 94
